# Sistemas de Recomendação - Algoritmo ALS

O algoritmo Alternating Least Squares (ALS) é uma técnica popular no campo de sistemas de recomendação, especialmente utilizado para a recomendação colaborativa baseada em fatores latentes. O princípio do ALS é decompor a matriz de interação usuário-item, que contém informações como avaliações de produtos ou interações de usuários com itens, em duas matrizes menores de fatores latentes - uma para usuários e outra para itens. Esses fatores latentes representam características subjetivas e ocultas que influenciam as preferências dos usuários e as propriedades dos itens.

O ALS funciona alternando entre a fixação de uma das matrizes de fatores latentes (usuários ou itens) e a otimização da outra, minimizando o erro quadrático das previsões comparado às avaliações conhecidas. Essa abordagem trata eficientemente os dados faltantes, que são comuns em sistemas de recomendação, e se adapta bem a grandes conjuntos de dados, sendo altamente escalável. O ALS é especialmente eficaz quando os dados são esparsos e há uma grande quantidade de usuários e itens, tornando-se uma escolha popular em plataformas de streaming, e-commerce e serviços de mídia social.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [2]:
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("postech") \
      .getOrCreate()

Vamos importar os dados

In [23]:
lines_raw = spark.read.text("data/movies.txt").rdd

Separando os valores

In [24]:
lines = lines_raw.map(lambda row: row.value.split("\t"))

Definindo o Schema pros nossos dados e criando um DataFrame com base neles

In [27]:
ratingsRDD = lines.map(lambda p: 
                        Row(userId=int(p[0]), 
                            movieId=int(p[1]), 
                            rating=float(p[2]), 
                            timestamp=int(p[3])))

In [30]:
ratings = spark.createDataFrame(ratingsRDD)
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



Vamos separar o nosso dataset em Treino e Teste

In [33]:
(train, test) = ratings.randomSplit([0.8, 0.2])

Vamos definir os hiperparâmetros do nosso modelo. A estratégia `drop` em *coldStartStrategy* significa que, caso o usuário tenha poucas interações com os itens em nossa base, ele será desconsiderado

In [34]:
model_als = ALS(maxIter=5, 
                regParam=0.01, 
                userCol="userId", 
                itemCol="movieId", 
                ratingCol="rating", 
                coldStartStrategy="drop")

Vamos treinar e avaliar o nosso modelo

In [35]:
model = model_als.fit(train)

In [39]:
predictions = model.transform(test)
predictions.show(5)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   148|    163|   4.0|877021402| 5.7215776|
|   148|    168|   5.0|877015900|  3.776377|
|   148|    169|   5.0|877020297|   3.79995|
|   148|    173|   5.0|877017054|   4.21437|
|   148|    214|   5.0|877019882|   3.46442|
+------+-------+------+---------+----------+
only showing top 5 rows



In [40]:
evaluator = RegressionEvaluator(metricName="rmse", 
                                labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("RMSE: " + str(rmse))

RMSE: 1.0752966335633836


Vamos gerar 10 recomendações para todos os usuários

In [41]:
userRec = model.recommendForAllUsers(10)

In [44]:
userRec.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{793, 5.9756246}...|
|     3|[{909, 7.3104134}...|
|     5|[{998, 6.3981543}...|
|     6|[{1643, 5.8951826...|
|     9|[{253, 12.308793}...|
|    12|[{1160, 7.9621253...|
|    13|[{464, 6.348189},...|
|    15|[{800, 7.9508986}...|
|    16|[{534, 7.257537},...|
|    17|[{1311, 7.617544}...|
|    19|[{1478, 9.177602}...|
|    20|[{984, 8.75906}, ...|
|    22|[{1129, 8.363482}...|
|    26|[{320, 5.305855},...|
|    27|[{1438, 7.5094438...|
|    28|[{1409, 5.64722},...|
|    31|[{967, 7.269172},...|
|    34|[{960, 11.1837635...|
|    35|[{253, 8.863161},...|
|    37|[{1154, 7.2954597...|
+------+--------------------+
only showing top 20 rows



Vamos fazer o inverso, vamos gerar 10 recomendações para todos os itens

In [45]:
movieRec = model.recommendForAllItems(10)

In [46]:
movieRec.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{282, 7.158555},...|
|      3|[{127, 8.492465},...|
|      6|[{180, 11.012345}...|
|     12|[{820, 7.4876337}...|
|     13|[{39, 8.128774}, ...|
|     16|[{820, 6.2807603}...|
|     20|[{39, 7.128742}, ...|
|     22|[{688, 6.7827554}...|
|     26|[{39, 7.1450114},...|
|     27|[{475, 9.555755},...|
|     28|[{688, 6.5880237}...|
|     31|[{688, 5.5656867}...|
|     34|[{202, 9.75335}, ...|
|     40|[{408, 9.0566025}...|
|     44|[{39, 9.685296}, ...|
|     47|[{820, 9.8515215}...|
|     52|[{310, 6.9967775}...|
|     53|[{39, 8.481455}, ...|
|     65|[{39, 7.5896506},...|
|     76|[{127, 6.9921494}...|
+-------+--------------------+
only showing top 20 rows



Vamos pegar os ids dos filmes recomendados para cada usuário

In [51]:
UserRecsItemId = userRec.select(userRec['userId'], 
                                userRec['recommendations']['movieId'].alias('movieId'))
UserRecsItemId.show(5, False)

+------+--------------------------------------------------------+
|userId|movieId                                                 |
+------+--------------------------------------------------------+
|1     |[793, 626, 543, 1142, 730, 169, 1143, 1367, 536, 853]   |
|3     |[909, 593, 1069, 954, 308, 1397, 574, 950, 641, 1428]   |
|5     |[998, 954, 1209, 916, 1206, 1159, 793, 613, 968, 1269]  |
|6     |[1643, 718, 1512, 1368, 573, 653, 610, 320, 1203, 493]  |
|9     |[253, 1265, 1113, 1245, 947, 219, 1062, 320, 1132, 1273]|
+------+--------------------------------------------------------+
only showing top 5 rows

